# Proto1

## Importataan tarvittavat kirjastot

from lightfm import LightFM
from lightfm.data import Dataset

import numpy as np
import pandas as pd

## Määritetään halutut ominaisuudet yrityksille

In [7]:
SELECTED_COMPANY_FEATURES = ['location_region_code', 'industry_code']

## Ladataan raakadata yrityksistä

In [9]:
COMPANIES_RAW = pd \
        .read_csv('data/prod_data_companies_2021_08_22.csv',
                  delimiter='\t',
                  dtype={
                      'business_id': 'string',
                      'company_name': 'string',
                      'company_form': 'string',
                      'company_form_code': 'string',
                      'location_region': 'string',
                      'location_region_code': 'string',
                      'location_municipality': 'string',
                      'location_municipality_code': 'string',
                      'industry_code': 'string',
                      'company_status': 'string',
                      'company_status_code': 'string',
                      'personnel_class': 'string'
                  }
                  )

## Käsitellään ja poimitaan halutut yritysdatat

In [12]:
ITEM_IDS = list(company_df['business_id'].values)

ITEM_FEATURE_LABELS = set() for feature in SELECTED_FEATURES: item_feature_labels |= set(company_df[feature].unique())
        
print(ITEM_FEATURE_LABELS)

SyntaxError: invalid syntax (385305549.py, line 3)